<a href="https://colab.research.google.com/github/sathindudezoysa/torrent_download/blob/main/Torrent_To_Google_Drive_Downloader_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Google Drive Downloader v2

### Mount Google Drive
To stream files we need to mount Google Drive.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Dependency
https://www.libtorrent.org/

In [3]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-libtorrent is already the newest version (2.0.5-5).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [14]:
import libtorrent as lt
import time
import datetime
import os

# Path to your .torrent file
torrent_file = "/content/drive/My Drive/films/torrent_file/FinalD.torrent"

# Step 1: Validate the torrent file
if not os.path.exists(torrent_file):
    print("Error: Torrent file not found.")
    exit(1)

try:
    info = lt.torrent_info(torrent_file)
    print("Torrent loaded successfully!")
    print("Torrent name:", info.name())
    print("Files in torrent:")
    for f in info.files():
        print(" -", f.path, "(", f.size, "bytes )")
except Exception as e:
    print("Failed to load torrent:", e)
    exit(1)

# Step 2: Set up the session
ses = lt.session()
ses.listen_on(6881, 6891)

# Add DHT routers for better peer discovery
ses.start_dht()
ses.add_dht_router("router.bittorrent.com", 6881)
ses.add_dht_router("router.utorrent.com", 6881)
ses.add_dht_router("dht.transmissionbt.com", 6881)

# Optional extensions for better compatibility
ses.add_extension('ut_metadata')
ses.add_extension('ut_pex')

# Step 3: Torrent parameters
params = {
    'save_path': '/content/drive/My Drive/films/',
    'storage_mode': lt.storage_mode_t.storage_mode_sparse,
    'ti': info,
}

# Step 4: Add torrent to session
handle = ses.add_torrent(params)
print(f"\nStarting download of: {handle.name()}")
begin = time.time()
print("Download started at:", datetime.datetime.now())

# Monitor the download
state_str = [
    'queued', 'checking', 'downloading metadata',
    'downloading', 'finished', 'seeding', 'allocating'
]

while not handle.status().is_seeding:
    s = handle.status()
    state_index = s.state
    state_label = state_str[state_index] if 0 <= state_index < len(state_str) else 'unknown'

    print('%.2f%% complete (down: %.1f kB/s up: %.1f kB/s peers: %d) %s' %
          (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
           s.num_peers, state_label))
    time.sleep(5)

# Completion summary
end = time.time()
print("\nDownload complete:", handle.name())
print("Elapsed Time: ", int((end - begin) // 60), "min :", int((end - begin) % 60), "sec")
print("Completed at:", datetime.datetime.now())


Torrent loaded successfully!
Torrent name: Final Destination Bloodlines (2025) [720p] [WEBRip] [YTS.MX]
Files in torrent:
 - Final Destination Bloodlines (2025) [720p] [WEBRip] [YTS.MX]/Final.Destination.Bloodlines.2025.720p.WEBRip.x264.AAC-[YTS.MX].mp4 ( 1057302518 bytes )
 - Final Destination Bloodlines (2025) [720p] [WEBRip] [YTS.MX]/Final.Destination.Bloodlines.2025.720p.WEBRip.x264.AAC-[YTS.MX].srt ( 96058 bytes )
 - Final Destination Bloodlines (2025) [720p] [WEBRip] [YTS.MX]/YTSYifyUP... (TOR).txt ( 604 bytes )
 - Final Destination Bloodlines (2025) [720p] [WEBRip] [YTS.MX]/www.YTS.MX.jpg ( 53226 bytes )
 - Final Destination Bloodlines (2025) [720p] [WEBRip] [YTS.MX]/Subs/English.srt ( 96058 bytes )
 - Final Destination Bloodlines (2025) [720p] [WEBRip] [YTS.MX]/Subs/kor.srt ( 85678 bytes )

Starting download of: Final Destination Bloodlines (2025) [720p] [WEBRip] [YTS.MX]
Download started at: 2025-06-20 04:11:40.308370
0.00% complete (down: 0.0 kB/s up: 0.0 kB/s peers: 0) unkno

### Code to download torrent
Variable **link** stores the link string.

In [8]:
import libtorrent as lt
import time
import datetime

ses = lt.session()
ses.listen_on(6881, 6891)
params = {
    'save_path': '/content/drive/My Drive/Torrent/',
    'storage_mode': lt.storage_mode_t(2),
    'paused': False,
    'auto_managed': True,
    'duplicate_is_error': True}
link = "https://yts.lt/torrent/download/866BDCFA006930A718ADBC21D8CAE1F2C7F9D8B3" # PASTE TORRENT/MAGNET LINK HERE
print(link)

handle = lt.add_magnet_uri(ses, link, params)
ses.start_dht()

begin = time.time()
print(datetime.datetime.now())

print ('Downloading Metadata...')
while (not handle.has_metadata()):
    time.sleep(1)
print ('Got Metadata, Starting Torrent Download...')

print("Starting", handle.name())

while (handle.status().state != lt.torrent_status.seeding):
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata', \
            'downloading', 'finished', 'seeding', 'allocating']
    print ('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
            (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
            s.num_peers, state_str[s.state]))
    time.sleep(5)

end = time.time()
print(handle.name(), "COMPLETE")

print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")

print(datetime.datetime.now())

ModuleNotFoundError: No module named 'libtorrent'